In [1]:
from Data_Processor import Data_Processor
from Data_Processor import clean
import numpy as np
import json

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/yunzehui/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/yunzehui/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yunzehui/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## predict sentiment

In [2]:
import time
import pandas as pd
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
import pickle

w2v_model = Word2Vec.load('/Users/yunzehui/Desktop/Morgan Stanley_Zehui/Sentiment_Prediction_Model-LSTM/model.w2v')
model = load_model('/Users/yunzehui/Desktop/Morgan Stanley_Zehui/Sentiment_Prediction_Model-LSTM/model.h5') 
with open('/Users/yunzehui/Desktop/Morgan Stanley_Zehui/Sentiment_Prediction_Model-LSTM/tokenizer.pkl', 'rb') as handle: tokenizer = pickle.load(handle) 
with open('/Users/yunzehui/Desktop/Morgan Stanley_Zehui/Sentiment_Prediction_Model-LSTM/encoder.pkl', 'rb') as handle: encoder = pickle.load(handle)

SEQUENCE_LENGTH = 300

Using TensorFlow backend.


In [70]:
# SENTIMENT
POSITIVE = "POSITIVE"
NEGATIVE = "NEGATIVE"
NEUTRAL = "NEUTRAL"
SENTIMENT_THRESHOLDS = (0.2, 0.8)

def decode_sentiment(score, include_neutral=True):
    if include_neutral:        
        label = NEUTRAL
        if score <= SENTIMENT_THRESHOLDS[0]:
            label = NEGATIVE
        elif score >= SENTIMENT_THRESHOLDS[1]:
            label = POSITIVE

        return label
    else:
        return NEGATIVE if score < 0.5 else POSITIVE
    
def predict(data, dl, include_neutral=True):
    for month in range(len(data)):
        for i in range(len(data[month])):
            text = data[month][i]['text']
            # Tokenize text
            x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=SEQUENCE_LENGTH)
            # Predict
            score = model.predict([x_test])[0]
            # Decode sentiment
            label = decode_sentiment(score, include_neutral=include_neutral)
            data[month][i]['label']=label
            data[month][i]['score']=float(score)
        print(dl[month], " completed")
    return data  

def datelist(start_month, end_month):
    start_year = int(start_month[:4])
    start_month = int(start_month[-2:])
    end_year = int(end_month[:4])
    end_month = int(end_month[-2:])
    if start_year == end_year:
        month_range = range(start_month, end_month + 1)
        date_list = ["{year}-{month:0=2d}".format(year=str(start_year), month=M) for M in month_range]
        return date_list
    year_range = range(start_year + 1, end_year)
    start_year_month_range = range(start_month, 13)
    end_year_month_range = range(1, end_month + 1)
    date_list = ["{year}-{month:0=2d}".format(year=str(start_year), month=M) for M in start_year_month_range]
    date_list += ["{year}-{month:0=2d}".format(year=str(Y), month=M) for Y in year_range for M in range(1, 13)]
    date_list += ["{year}-{month:0=2d}".format(year=str(end_year), month=M) for M in end_year_month_range]
    return date_list

# Discount Brokerage

## Charles Schwab

In [68]:
start_month='2017-06'
end_month='2020-05'

DP=Data_Processor(start_month, end_month,
                  template=["Data/Discount Brokerage/Charles Schwab/CharlesSchwab"])
DP.readdata()
print(DP.datanums())
DP.specifylang()
DP.removenoise() 
DP.clean()
print(DP.datanums())

([2026, 2120, 1899, 2017, 2833, 3110, 2667, 2326, 2215, 2359, 3729, 2609, 2391, 2318, 2011, 2208, 3107, 2365, 1789, 2306, 2434, 2125, 2900, 4349, 2703, 2411, 1863, 2406, 5287, 4270, 2641, 2169, 2685, 2829, 3150, 2841], 95468)
([725, 890, 801, 901, 1109, 1390, 1032, 1067, 1088, 1134, 2435, 1469, 1178, 1084, 976, 1259, 1625, 1218, 929, 957, 1218, 1085, 1664, 2457, 2022, 1355, 1052, 1167, 2977, 2445, 1571, 1281, 1767, 1869, 2006, 1870], 51073)


In [45]:
len(DP.data)#共有多少个月的数据
DP.data[0] #第一个月数据
len(DP.data[0])#第一个月内有多少条tweets
#给第一个月第一条数据赋值
DP.data[0][0]['label']='Positive'
DP.data[0][0]['score']=12

In [48]:
DP.data

[[{'has_media': True,
   'hashtags': ['Fed'],
   'img_urls': ['https://pbs.twimg.com/media/DCE5DpAVwAEtJ_a.jpg'],
   'is_replied': False,
   'is_reply_to': False,
   'likes': 1,
   'links': ['http://www.schwab.com/insights/market-commentary/interpreting-federal-reserve-communications'],
   'parent_tweet_id': '',
   'replies': 0,
   'reply_to_users': [],
   'retweets': 0,
   'screen_name': 'LSilverthorneCS',
   'text': 'Great article by CharlesSchwab strategist KathyJones How to interpret communications ',
   'text_html': '<p class="TweetTextSize js-tweet-text tweet-text" data-aria-label-part="0" lang="en">Great article by <a class="twitter-atreply pretty-link js-nav" data-mentioned-user-id="132703700" dir="ltr" href="/CharlesSchwab"><s>@</s><b><strong>CharlesSchwab</strong></b></a> strategist <a class="twitter-atreply pretty-link js-nav" data-mentioned-user-id="2840691561" dir="ltr" href="/KathyJones"><s>@</s><b>KathyJones</b></a>: How to interpret <a class="twitter-hashtag pretty-link

In [71]:
dl = datelist(start_month, end_month)
filepath=[]
for i in range(len(dl)):
    pathi = 'result0623/Discount Brokerage/Charles Schwab/CharlesSchwab'+dl[i]+'.json'
    filepath.append(pathi)

In [72]:
result = predict(DP.data, dl)

2017-06  completed
2017-07  completed
2017-08  completed
2017-09  completed
2017-10  completed
2017-11  completed
2017-12  completed
2018-01  completed
2018-02  completed
2018-03  completed
2018-04  completed
2018-05  completed
2018-06  completed
2018-07  completed
2018-08  completed
2018-09  completed
2018-10  completed
2018-11  completed
2018-12  completed
2019-01  completed
2019-02  completed
2019-03  completed
2019-04  completed
2019-05  completed
2019-06  completed
2019-07  completed
2019-08  completed
2019-09  completed
2019-10  completed
2019-11  completed
2019-12  completed
2020-01  completed
2020-02  completed
2020-03  completed
2020-04  completed
2020-05  completed


In [73]:
# run 1 time
for i in range(len(result)):
    with open(filepath[i],"w") as f:
        json.dump(result[i],f)

## eTrade

In [74]:
start_month='2017-06'
end_month='2020-05'

DP=Data_Processor(start_month, end_month,
                  template=["Data/Discount Brokerage/eTrade/eTrade"])
DP.readdata()
print(DP.datanums())
DP.specifylang()
DP.removenoise() 
DP.clean()
print(DP.datanums())

([2168, 1786, 1947, 2240, 2134, 3977, 2477, 2953, 2998, 2718, 3458, 1953, 1833, 1778, 1656, 1830, 1732, 1890, 2060, 2132, 1699, 1930, 2926, 2302, 1917, 1868, 2157, 1618, 3585, 1962, 1579, 2283, 3912, 5184, 3355, 3863], 87860)
([1435, 1351, 1533, 1642, 1527, 3120, 1592, 2052, 2364, 1600, 2645, 1364, 1350, 1235, 1134, 1217, 1285, 1371, 1576, 1603, 1229, 1488, 1893, 1907, 1551, 1440, 1824, 1253, 2730, 1601, 1265, 1846, 2536, 4602, 2844, 3278], 66283)


In [77]:
dl = datelist(start_month, end_month)
filepath=[]
for i in range(len(dl)):
    pathi = 'result0623/Discount Brokerage/eTrade/eTrade'+dl[i]+'.json'
    filepath.append(pathi)

In [78]:
result = predict(DP.data, dl)

2017-06  completed
2017-07  completed
2017-08  completed
2017-09  completed
2017-10  completed
2017-11  completed
2017-12  completed
2018-01  completed
2018-02  completed
2018-03  completed
2018-04  completed
2018-05  completed
2018-06  completed
2018-07  completed
2018-08  completed
2018-09  completed
2018-10  completed
2018-11  completed
2018-12  completed
2019-01  completed
2019-02  completed
2019-03  completed
2019-04  completed
2019-05  completed
2019-06  completed
2019-07  completed
2019-08  completed
2019-09  completed
2019-10  completed
2019-11  completed
2019-12  completed
2020-01  completed
2020-02  completed
2020-03  completed
2020-04  completed
2020-05  completed


In [79]:
# run 1 time
for i in range(len(result)):
    with open(filepath[i],"w") as f:
        json.dump(result[i],f)

2017-06  completed
2017-07  completed
2017-08  completed
2017-09  completed
2017-10  completed
2017-11  completed
2017-12  completed
2018-01  completed
2018-02  completed
2018-03  completed
2018-04  completed
2018-05  completed
2018-06  completed
2018-07  completed
2018-08  completed
2018-09  completed
2018-10  completed
2018-11  completed
2018-12  completed
2019-01  completed
2019-02  completed
2019-03  completed
2019-04  completed
2019-05  completed
2019-06  completed
2019-07  completed
2019-08  completed
2019-09  completed
2019-10  completed
2019-11  completed
2019-12  completed
2020-01  completed
2020-02  completed
2020-03  completed
2020-04  completed
2020-05  completed


## Fidelity

In [80]:
start_month='2017-06'
end_month='2020-05'

DP=Data_Processor(start_month, end_month,
                  template=["Data/Discount Brokerage/Fidelity/FidelityInvestments"])
DP.readdata()
print(DP.datanums())
DP.specifylang()
DP.removenoise() 
DP.clean()
print(DP.datanums())

([746, 774, 823, 644, 829, 743, 897, 690, 832, 810, 694, 890, 1336, 962, 1234, 1648, 1809, 1111, 1054, 1107, 1411, 1026, 954, 1747, 772, 829, 937, 872, 1311, 973, 877, 934, 915, 635, 579, 431], 34836)
([143, 159, 217, 171, 273, 194, 227, 237, 328, 333, 267, 280, 335, 241, 289, 415, 503, 235, 285, 473, 445, 335, 289, 565, 275, 244, 323, 364, 534, 446, 356, 377, 392, 280, 309, 234], 11373)


In [81]:
dl = datelist(start_month, end_month)
filepath=[]
for i in range(len(dl)):
    pathi = 'result0623/Discount Brokerage/Fidelity/FidelityInvestments'+dl[i]+'.json'
    filepath.append(pathi)

In [82]:
result = predict(DP.data,dl)

2017-06  completed
2017-07  completed
2017-08  completed
2017-09  completed
2017-10  completed
2017-11  completed
2017-12  completed
2018-01  completed
2018-02  completed
2018-03  completed
2018-04  completed
2018-05  completed
2018-06  completed
2018-07  completed
2018-08  completed
2018-09  completed
2018-10  completed
2018-11  completed
2018-12  completed
2019-01  completed
2019-02  completed
2019-03  completed
2019-04  completed
2019-05  completed
2019-06  completed
2019-07  completed
2019-08  completed
2019-09  completed
2019-10  completed
2019-11  completed
2019-12  completed
2020-01  completed
2020-02  completed
2020-03  completed
2020-04  completed
2020-05  completed


In [83]:
# run 1 time
for i in range(len(result)):
    with open(filepath[i],"w") as f:
        json.dump(result[i],f)

2017-06  completed
2017-07  completed
2017-08  completed
2017-09  completed
2017-10  completed
2017-11  completed
2017-12  completed
2018-01  completed
2018-02  completed
2018-03  completed
2018-04  completed
2018-05  completed
2018-06  completed
2018-07  completed
2018-08  completed
2018-09  completed
2018-10  completed
2018-11  completed
2018-12  completed
2019-01  completed
2019-02  completed
2019-03  completed
2019-04  completed
2019-05  completed
2019-06  completed
2019-07  completed
2019-08  completed
2019-09  completed
2019-10  completed
2019-11  completed
2019-12  completed
2020-01  completed
2020-02  completed
2020-03  completed
2020-04  completed
2020-05  completed


## Vanguard

In [84]:
start_month='2017-06'
end_month='2020-05'

DP=Data_Processor(start_month, end_month,
                  template=["Data/Discount Brokerage/Vanguard/Vanguard"])
DP.readdata()
print(DP.datanums())
DP.specifylang()
DP.removenoise() 
DP.clean()
print(DP.datanums())

([10217, 10183, 10194, 10214, 10148, 10207, 10186, 10135, 9714, 10171, 10221, 10229, 10180, 10198, 10217, 10213, 10223, 10210, 10209, 10142, 10139, 10228, 10189, 10159, 10184, 10189, 10201, 10170, 10061, 10208, 10183, 10183, 9870, 10249, 10144, 10185], 365953)
([3242, 3779, 4006, 3721, 3855, 3983, 3404, 3496, 3522, 4050, 4291, 4725, 4800, 4967, 4612, 4817, 4585, 4817, 4487, 4560, 4310, 4305, 4040, 3862, 4413, 4725, 4745, 4225, 4962, 5276, 5122, 5411, 5213, 5011, 5561, 5031], 159931)


In [85]:
dl = datelist(start_month, end_month)
filepath=[]
for i in range(len(dl)):
    pathi = 'result0623/Discount Brokerage/Vanguard/Vanguard'+dl[i]+'.json'
    filepath.append(pathi)

In [86]:
result = predict(DP.data,dl)

2017-06  completed
2017-07  completed
2017-08  completed
2017-09  completed
2017-10  completed
2017-11  completed
2017-12  completed
2018-01  completed
2018-02  completed
2018-03  completed
2018-04  completed
2018-05  completed
2018-06  completed
2018-07  completed
2018-08  completed
2018-09  completed
2018-10  completed
2018-11  completed
2018-12  completed
2019-01  completed
2019-02  completed
2019-03  completed
2019-04  completed
2019-05  completed
2019-06  completed
2019-07  completed
2019-08  completed
2019-09  completed
2019-10  completed
2019-11  completed
2019-12  completed
2020-01  completed
2020-02  completed
2020-03  completed
2020-04  completed
2020-05  completed


In [87]:
# run 1 time
for i in range(len(result)):
    with open(filepath[i],"w") as f:
        json.dump(result[i],f)

2017-06  completed
2017-07  completed
2017-08  completed
2017-09  completed
2017-10  completed
2017-11  completed
2017-12  completed
2018-01  completed
2018-02  completed
2018-03  completed
2018-04  completed
2018-05  completed
2018-06  completed
2018-07  completed
2018-08  completed
2018-09  completed
2018-10  completed
2018-11  completed
2018-12  completed
2019-01  completed
2019-02  completed
2019-03  completed
2019-04  completed
2019-05  completed
2019-06  completed
2019-07  completed
2019-08  completed
2019-09  completed
2019-10  completed
2019-11  completed
2019-12  completed
2020-01  completed
2020-02  completed
2020-03  completed
2020-04  completed
2020-05  completed


## TD Ameritrade

In [88]:
start_month='2017-06'
end_month='2020-05'

DP=Data_Processor(start_month, end_month,
                  template=["Data/Discount Brokerage/TD Ameritrade/TDAmeritrade"])
DP.readdata()
print(DP.datanums())
DP.specifylang()
DP.removenoise() 
DP.clean()
print(DP.datanums())
#,"Data/Discount Brokerage/eTrade/eTrade","Data/Discount Brokerage/Fidelity/FidelityInvestments","Data/Discount Brokerage/Vanguard/Vanguard","Data/Discount Brokerage/TD Ameritrade/TDAmeritrade"

([3879, 2475, 4056, 2672, 3472, 2528, 4949, 4303, 4671, 4371, 4123, 3619, 4834, 3020, 3003, 3569, 4988, 2788, 3859, 3680, 2912, 3058, 4438, 3910, 4356, 3742, 2862, 2619, 4674, 4588, 3323, 4008, 3700, 6366, 5515, 5675], 140605)
([2699, 1204, 2008, 1298, 1763, 1323, 2188, 2028, 2768, 2362, 2018, 1810, 2895, 1518, 1727, 2059, 2506, 1614, 2378, 2037, 1725, 1859, 2535, 2425, 3029, 1892, 1872, 1630, 2777, 2705, 1987, 2543, 2469, 4650, 3531, 3691], 81523)


In [89]:
dl = datelist(start_month, end_month)
filepath=[]
for i in range(len(dl)):
    pathi = 'result0623/Discount Brokerage/TD Ameritrade/TDAmeritrade'+dl[i]+'.json'
    filepath.append(pathi)

In [90]:
result = predict(DP.data,dl)

2017-06  completed
2017-07  completed
2017-08  completed
2017-09  completed
2017-10  completed
2017-11  completed
2017-12  completed
2018-01  completed
2018-02  completed
2018-03  completed
2018-04  completed
2018-05  completed
2018-06  completed
2018-07  completed
2018-08  completed
2018-09  completed
2018-10  completed
2018-11  completed
2018-12  completed
2019-01  completed
2019-02  completed
2019-03  completed
2019-04  completed
2019-05  completed
2019-06  completed
2019-07  completed
2019-08  completed
2019-09  completed
2019-10  completed
2019-11  completed
2019-12  completed
2020-01  completed
2020-02  completed
2020-03  completed
2020-04  completed
2020-05  completed


In [91]:
# run 1 time
for i in range(len(result)):
    with open(filepath[i],"w") as f:
        json.dump(result[i],f)

2017-06  completed
2017-07  completed
2017-08  completed
2017-09  completed
2017-10  completed
2017-11  completed
2017-12  completed
2018-01  completed
2018-02  completed
2018-03  completed
2018-04  completed
2018-05  completed
2018-06  completed
2018-07  completed
2018-08  completed
2018-09  completed
2018-10  completed
2018-11  completed
2018-12  completed
2019-01  completed
2019-02  completed
2019-03  completed
2019-04  completed
2019-05  completed
2019-06  completed
2019-07  completed
2019-08  completed
2019-09  completed
2019-10  completed
2019-11  completed
2019-12  completed
2020-01  completed
2020-02  completed
2020-03  completed
2020-04  completed
2020-05  completed


# Robo Advisor

## wealthfront

In [92]:
start_month='2017-06'
end_month='2020-05'

DP=Data_Processor(start_month, end_month,
                  template=["Data/Robo Advisor/wealthfront/Wealthfront"])
DP.readdata()
print(DP.datanums())
DP.specifylang()
DP.removenoise() 
DP.clean()
print(DP.datanums())

([1945, 1364, 940, 854, 1215, 1120, 931, 1746, 1895, 1338, 892, 1156, 780, 850, 813, 558, 1043, 532, 1698, 724, 978, 913, 780, 751, 887, 1098, 892, 767, 585, 583, 1012, 760, 744, 1018, 805, 588], 35555)
([792, 718, 428, 363, 582, 498, 409, 678, 892, 731, 447, 642, 337, 430, 402, 277, 591, 283, 820, 462, 514, 488, 510, 366, 536, 683, 638, 479, 415, 404, 768, 577, 548, 744, 550, 384], 19386)


In [93]:
dl = datelist(start_month, end_month)
filepath=[]
for i in range(len(dl)):
    pathi = 'result0623/Robo Advisor/wealthfront/Wealthfront'+dl[i]+'.json'
    filepath.append(pathi)

In [94]:
result = predict(DP.data,dl)

2017-06  completed
2017-07  completed
2017-08  completed
2017-09  completed
2017-10  completed
2017-11  completed
2017-12  completed
2018-01  completed
2018-02  completed
2018-03  completed
2018-04  completed
2018-05  completed
2018-06  completed
2018-07  completed
2018-08  completed
2018-09  completed
2018-10  completed
2018-11  completed
2018-12  completed
2019-01  completed
2019-02  completed
2019-03  completed
2019-04  completed
2019-05  completed
2019-06  completed
2019-07  completed
2019-08  completed
2019-09  completed
2019-10  completed
2019-11  completed
2019-12  completed
2020-01  completed
2020-02  completed
2020-03  completed
2020-04  completed
2020-05  completed


In [95]:
# run 1 time
for i in range(len(result)):
    with open(filepath[i],"w") as f:
        json.dump(result[i],f)

2017-06  completed
2017-07  completed
2017-08  completed
2017-09  completed
2017-10  completed
2017-11  completed
2017-12  completed
2018-01  completed
2018-02  completed
2018-03  completed
2018-04  completed
2018-05  completed
2018-06  completed
2018-07  completed
2018-08  completed
2018-09  completed
2018-10  completed
2018-11  completed
2018-12  completed
2019-01  completed
2019-02  completed
2019-03  completed
2019-04  completed
2019-05  completed
2019-06  completed
2019-07  completed
2019-08  completed
2019-09  completed
2019-10  completed
2019-11  completed
2019-12  completed
2020-01  completed
2020-02  completed
2020-03  completed
2020-04  completed
2020-05  completed


## Personal Capital

In [96]:
start_month='2017-06'
end_month='2020-05'

DP=Data_Processor(start_month, end_month,
                  template=["Data/Robo Advisor/Personal Capital/PersonalCapital"])
DP.readdata()
print(DP.datanums())
DP.specifylang()
DP.removenoise() 
DP.clean()
print(DP.datanums())

([591, 730, 995, 817, 607, 971, 1031, 965, 833, 784, 929, 816, 855, 716, 759, 778, 872, 650, 747, 323, 907, 887, 737, 582, 795, 733, 705, 607, 759, 638, 618, 783, 780, 659, 650, 701], 27310)
([343, 308, 440, 438, 330, 481, 442, 475, 485, 424, 511, 339, 445, 409, 415, 386, 400, 373, 413, 170, 435, 461, 459, 353, 469, 458, 458, 387, 578, 461, 446, 524, 573, 454, 496, 520], 15559)


In [97]:
dl = datelist(start_month, end_month)
filepath=[]
for i in range(len(dl)):
    pathi = 'result0623/Robo Advisor/Personal Capital/PersonalCapital'+dl[i]+'.json'
    filepath.append(pathi)

In [98]:
result = predict(DP.data,dl)

2017-06  completed
2017-07  completed
2017-08  completed
2017-09  completed
2017-10  completed
2017-11  completed
2017-12  completed
2018-01  completed
2018-02  completed
2018-03  completed
2018-04  completed
2018-05  completed
2018-06  completed
2018-07  completed
2018-08  completed
2018-09  completed
2018-10  completed
2018-11  completed
2018-12  completed
2019-01  completed
2019-02  completed
2019-03  completed
2019-04  completed
2019-05  completed
2019-06  completed
2019-07  completed
2019-08  completed
2019-09  completed
2019-10  completed
2019-11  completed
2019-12  completed
2020-01  completed
2020-02  completed
2020-03  completed
2020-04  completed
2020-05  completed


In [99]:
# run 1 time
for i in range(len(result)):
    with open(filepath[i],"w") as f:
        json.dump(result[i],f)

## Betterment

In [100]:
start_month='2017-06'
end_month='2020-05'

DP=Data_Processor(start_month, end_month,
                  template=["Data/Robo Advisor/Betterment/Betterment"])
DP.readdata()
print(DP.datanums())
DP.specifylang()
DP.removenoise() 
DP.clean()
print(DP.datanums())

([16, 18, 16, 14, 6, 34, 6, 36, 54, 28, 56, 144, 142, 154, 36, 78, 54, 80, 80, 46, 22, 50, 74, 26, 44, 50, 50, 62, 60, 50, 36, 24, 38, 58, 44, 30], 1816)
([4, 18, 14, 12, 6, 24, 4, 20, 16, 22, 20, 30, 34, 38, 26, 66, 20, 16, 14, 38, 12, 34, 50, 12, 28, 22, 34, 36, 38, 28, 18, 14, 24, 32, 22, 24], 870)


In [101]:
dl = datelist(start_month, end_month)
filepath=[]
for i in range(len(dl)):
    pathi = 'result0623/Robo Advisor/Betterment/Betterment'+dl[i]+'.json'
    filepath.append(pathi)

In [102]:
result = predict(DP.data,dl)

2017-06  completed
2017-07  completed
2017-08  completed
2017-09  completed
2017-10  completed
2017-11  completed
2017-12  completed
2018-01  completed
2018-02  completed
2018-03  completed
2018-04  completed
2018-05  completed
2018-06  completed
2018-07  completed
2018-08  completed
2018-09  completed
2018-10  completed
2018-11  completed
2018-12  completed
2019-01  completed
2019-02  completed
2019-03  completed
2019-04  completed
2019-05  completed
2019-06  completed
2019-07  completed
2019-08  completed
2019-09  completed
2019-10  completed
2019-11  completed
2019-12  completed
2020-01  completed
2020-02  completed
2020-03  completed
2020-04  completed
2020-05  completed


In [103]:
# run 1 time
for i in range(len(result)):
    with open(filepath[i],"w") as f:
        json.dump(result[i],f)

## Robo

In [104]:
start_month='2017-06'
end_month='2020-05'

DP=Data_Processor(start_month, end_month,
                  template=["Data/Robo Advisor/Robo/RoboAdvisor"])
DP.readdata()
print(DP.datanums())
DP.specifylang()
DP.removenoise() 
DP.clean()
print(DP.datanums())

([4292, 4870, 5287, 5939, 5613, 5857, 5157, 5262, 5435, 5168, 4583, 2856, 2793, 2599, 2515, 2031, 2419, 2385, 3433, 3143, 2327, 3151, 2882, 2509, 1787, 1884, 1586, 1858, 1688, 1365, 1321, 1380, 1381, 1452, 1362, 1599], 111169)
([1247, 1441, 1609, 1644, 1483, 1373, 1246, 1528, 1823, 1268, 985, 877, 915, 941, 867, 776, 1024, 792, 751, 748, 621, 711, 531, 552, 676, 724, 626, 667, 680, 603, 540, 562, 579, 606, 534, 621], 33171)


In [105]:
dl = datelist(start_month, end_month)
filepath=[]
for i in range(len(dl)):
    pathi = 'result0623/Robo Advisor/Robo/Robo'+dl[i]+'.json'
    filepath.append(pathi)

In [106]:
result = predict(DP.data,dl)

2017-06  completed
2017-07  completed
2017-08  completed
2017-09  completed
2017-10  completed
2017-11  completed
2017-12  completed
2018-01  completed
2018-02  completed
2018-03  completed
2018-04  completed
2018-05  completed
2018-06  completed
2018-07  completed
2018-08  completed
2018-09  completed
2018-10  completed
2018-11  completed
2018-12  completed
2019-01  completed
2019-02  completed
2019-03  completed
2019-04  completed
2019-05  completed
2019-06  completed
2019-07  completed
2019-08  completed
2019-09  completed
2019-10  completed
2019-11  completed
2019-12  completed
2020-01  completed
2020-02  completed
2020-03  completed
2020-04  completed
2020-05  completed


In [107]:
# run 1 time
for i in range(len(result)):
    with open(filepath[i],"w") as f:
        json.dump(result[i],f)

# Bank Brokerage

## Merrill Lynch

In [108]:
start_month='2017-06'
end_month='2020-05'

DP=Data_Processor(start_month, end_month,
                  template=["Data/Bank Brokerage/Merrill Lynch/MerillLynch"])
DP.readdata()
print(DP.datanums())
DP.specifylang()
DP.removenoise() 
DP.clean()
print(DP.datanums())

([2347, 1732, 2042, 2253, 2432, 2340, 1770, 3481, 1990, 1215, 1821, 2469, 2286, 1694, 2236, 2180, 1958, 1725, 1547, 1722, 2698, 2079, 1820, 2302, 2710, 1954, 2102, 2397, 2073, 2193, 2262, 2334, 1942, 1792, 1760, 1677], 75335)
([662, 545, 653, 585, 579, 597, 593, 924, 624, 421, 701, 705, 787, 635, 815, 853, 738, 620, 560, 667, 816, 726, 695, 681, 842, 509, 613, 678, 593, 711, 643, 511, 565, 671, 535, 473], 23526)


In [109]:
dl = datelist(start_month, end_month)
filepath=[]
for i in range(len(dl)):
    pathi = 'result0623/Bank Brokerage/Merrill Lynch/MerrillLynch'+dl[i]+'.json'
    filepath.append(pathi)

In [110]:
result = predict(DP.data,dl)

2017-06  completed
2017-07  completed
2017-08  completed
2017-09  completed
2017-10  completed
2017-11  completed
2017-12  completed
2018-01  completed
2018-02  completed
2018-03  completed
2018-04  completed
2018-05  completed
2018-06  completed
2018-07  completed
2018-08  completed
2018-09  completed
2018-10  completed
2018-11  completed
2018-12  completed
2019-01  completed
2019-02  completed
2019-03  completed
2019-04  completed
2019-05  completed
2019-06  completed
2019-07  completed
2019-08  completed
2019-09  completed
2019-10  completed
2019-11  completed
2019-12  completed
2020-01  completed
2020-02  completed
2020-03  completed
2020-04  completed
2020-05  completed


In [111]:
# run 1 time
for i in range(len(result)):
    with open(filepath[i],"w") as f:
        json.dump(result[i],f)

## BOA

In [112]:
start_month='2017-06'
end_month='2020-05'

DP=Data_Processor(start_month, end_month,
                  template=["Data/Bank Brokerage/BOA/BankofAmerica"])
DP.readdata()
print(DP.datanums())
DP.specifylang()
DP.removenoise() 
DP.clean()
print(DP.datanums())

([10064, 10245, 9899, 10158, 10087, 9884, 10173, 9714, 10210, 9907, 9835, 10009, 9974, 9930, 10206, 10125, 9922, 9918, 10087, 9530, 10053, 9989, 9987, 8677, 8860, 9688, 9877, 10080, 10040, 9894, 10145, 9899, 10176, 10137, 10248, 10011], 357638)
([4711, 4050, 4020, 4394, 4723, 4956, 4968, 4687, 4927, 4650, 5421, 5739, 6188, 5397, 5685, 5760, 5533, 5751, 6441, 5817, 5824, 6703, 6407, 5908, 6495, 6770, 7015, 7378, 7107, 7544, 7426, 6578, 6620, 7300, 8139, 7187], 214219)


In [113]:
dl = datelist(start_month, end_month)
filepath=[]
for i in range(len(dl)):
    pathi = 'result0623/Bank Brokerage/BOA/BOA'+dl[i]+'.json'
    filepath.append(pathi)

In [114]:
result = predict(DP.data,dl)

2017-06  completed
2017-07  completed
2017-08  completed
2017-09  completed
2017-10  completed
2017-11  completed
2017-12  completed
2018-01  completed
2018-02  completed
2018-03  completed
2018-04  completed
2018-05  completed
2018-06  completed
2018-07  completed
2018-08  completed
2018-09  completed
2018-10  completed
2018-11  completed
2018-12  completed
2019-01  completed
2019-02  completed
2019-03  completed
2019-04  completed
2019-05  completed
2019-06  completed
2019-07  completed
2019-08  completed
2019-09  completed
2019-10  completed
2019-11  completed
2019-12  completed
2020-01  completed
2020-02  completed
2020-03  completed
2020-04  completed
2020-05  completed


In [115]:
# run 1 time
for i in range(len(result)):
    with open(filepath[i],"w") as f:
        json.dump(result[i],f)

## JPMorgan

In [116]:
start_month='2017-06'
end_month='2020-05'

DP=Data_Processor(start_month, end_month,
                  template=["Data/Bank Brokerage/JPMorgan/JPMorgan"])
DP.readdata()
print(DP.datanums())
DP.specifylang()
DP.removenoise() 
DP.clean()
print(DP.datanums())

([9792, 9837, 9446, 9984, 10073, 10053, 9956, 10036, 10186, 9916, 9816, 9938, 9461, 9563, 9860, 10203, 9942, 10052, 9823, 9783, 10201, 10194, 10045, 10251, 9576, 9926, 9689, 9770, 9745, 9351, 8618, 9710, 9293, 10218, 10181, 10189], 354677)
([3011, 3021, 3262, 3189, 3140, 3239, 2815, 3367, 3383, 3439, 3290, 3247, 3402, 3064, 2911, 3301, 3597, 3280, 3754, 3604, 3356, 3998, 4333, 4190, 4307, 4588, 4130, 4559, 4723, 4798, 4129, 4596, 4643, 4689, 4841, 4845], 136041)


In [117]:
dl = datelist(start_month, end_month)
filepath=[]
for i in range(len(dl)):
    pathi = 'result0623/Bank Brokerage/JPMorgan/JPMorgan'+dl[i]+'.json'
    filepath.append(pathi)

In [118]:
result = predict(DP.data,dl)

2017-06  completed
2017-07  completed
2017-08  completed
2017-09  completed
2017-10  completed
2017-11  completed
2017-12  completed
2018-01  completed
2018-02  completed
2018-03  completed
2018-04  completed
2018-05  completed
2018-06  completed
2018-07  completed
2018-08  completed
2018-09  completed
2018-10  completed
2018-11  completed
2018-12  completed
2019-01  completed
2019-02  completed
2019-03  completed
2019-04  completed
2019-05  completed
2019-06  completed
2019-07  completed
2019-08  completed
2019-09  completed
2019-10  completed
2019-11  completed
2019-12  completed
2020-01  completed
2020-02  completed
2020-03  completed
2020-04  completed
2020-05  completed


In [119]:
# run 1 time
for i in range(len(result)):
    with open(filepath[i],"w") as f:
        json.dump(result[i],f)

## Wells Fargo

In [132]:
start_month='2017-06'
end_month='2020-05'

DP=Data_Processor(start_month, end_month,
                  template=["Data/Bank Brokerage/Wells Fargo/WellsFargo"])
DP.readdata()
print(DP.datanums())
DP.specifylang()
DP.removenoise() 
DP.clean()
print(DP.datanums())

([10217, 10260, 10257, 9991, 10206, 10213, 10223, 10215, 10203, 10156, 10111, 9648, 9918, 10217, 10221, 10168, 10214, 10170, 9951, 10164, 10208, 10185, 10143, 10197, 10049, 10171, 10082, 10000, 10217, 10199, 9541, 10128, 10220, 10184, 10145, 10144], 364536)
([4692, 4458, 4472, 5095, 5013, 5573, 5490, 5786, 5343, 5541, 5812, 5390, 6011, 6235, 5951, 6295, 6343, 6167, 5962, 6353, 6992, 6541, 6980, 7184, 7520, 7658, 7241, 7210, 7008, 7665, 7213, 6723, 6670, 7912, 8150, 7736], 228385)


In [133]:
dl = datelist(start_month, end_month)
filepath=[]
for i in range(len(dl)):
    pathi = 'result0623/Bank Brokerage/Wells Fargo/WellsFargo'+dl[i]+'.json'
    filepath.append(pathi)

In [134]:
result = predict(DP.data,dl)

2017-06  completed
2017-07  completed
2017-08  completed
2017-09  completed
2017-10  completed
2017-11  completed
2017-12  completed
2018-01  completed
2018-02  completed
2018-03  completed
2018-04  completed
2018-05  completed
2018-06  completed
2018-07  completed
2018-08  completed
2018-09  completed
2018-10  completed
2018-11  completed
2018-12  completed
2019-01  completed
2019-02  completed
2019-03  completed
2019-04  completed
2019-05  completed
2019-06  completed
2019-07  completed
2019-08  completed
2019-09  completed
2019-10  completed
2019-11  completed
2019-12  completed
2020-01  completed
2020-02  completed
2020-03  completed
2020-04  completed
2020-05  completed


In [135]:
# run 1 time
for i in range(len(result)):
    with open(filepath[i],"w") as f:
        json.dump(result[i],f)

# Full Brokerage

## Goldman_Sachs

In [120]:
start_month='2017-06'
end_month='2020-05'

DP=Data_Processor(start_month, end_month,
                  template=["Data/Full Brokerage/Goldman_Sachs/Goldman_Sachs"])
DP.readdata()
print(DP.datanums())
DP.specifylang()
DP.removenoise() 
DP.clean()
print(DP.datanums())

([10243, 10220, 10187, 10231, 10127, 10214, 10201, 10190, 10188, 10201, 9984, 10158, 9891, 9917, 10016, 9853, 10058, 10125, 10185, 9951, 9983, 9906, 9783, 9835, 9738, 9698, 9790, 9917, 9698, 10061, 9527, 9962, 10090, 10093, 10191, 10130], 360542)
([3671, 4119, 4222, 4497, 4097, 4071, 4294, 4410, 4184, 3662, 3712, 3579, 3966, 4175, 4242, 3851, 4130, 4101, 4241, 4176, 3854, 4413, 4506, 4411, 4338, 4275, 4779, 5040, 5145, 5544, 5015, 4765, 5571, 5523, 5439, 5347], 159365)


In [121]:
dl = datelist(start_month, end_month)
filepath=[]
for i in range(len(dl)):
    pathi = 'result0623/Full Brokerage/Goldman_Sachs/Goldman_Sachs'+dl[i]+'.json'
    filepath.append(pathi)

In [122]:
result = predict(DP.data,dl)

2017-06  completed
2017-07  completed
2017-08  completed
2017-09  completed
2017-10  completed
2017-11  completed
2017-12  completed
2018-01  completed
2018-02  completed
2018-03  completed
2018-04  completed
2018-05  completed
2018-06  completed
2018-07  completed
2018-08  completed
2018-09  completed
2018-10  completed
2018-11  completed
2018-12  completed
2019-01  completed
2019-02  completed
2019-03  completed
2019-04  completed
2019-05  completed
2019-06  completed
2019-07  completed
2019-08  completed
2019-09  completed
2019-10  completed
2019-11  completed
2019-12  completed
2020-01  completed
2020-02  completed
2020-03  completed
2020-04  completed
2020-05  completed


In [123]:
# run 1 time
for i in range(len(result)):
    with open(filepath[i],"w") as f:
        json.dump(result[i],f)

## Morgan_Stanley

In [124]:
start_month='2017-06'
end_month='2020-05'

DP=Data_Processor(start_month, end_month,
                  template=["Data/Full Brokerage/Morgan_Stanley/Morgan_Stanley"])
DP.readdata()
print(DP.datanums())
DP.specifylang()
DP.removenoise() 
DP.clean()
print(DP.datanums())

([9513, 9768, 9885, 9534, 9371, 9661, 10183, 9527, 9292, 9290, 9059, 9399, 9927, 9314, 9829, 9700, 9283, 9411, 9914, 9075, 8763, 9295, 8949, 9234, 8468, 8648, 9049, 8523, 9401, 8371, 7937, 8180, 8683, 9569, 8916, 9644], 332565)
([2619, 2982, 3456, 3206, 3040, 3505, 3968, 3319, 2931, 3106, 2701, 3589, 3197, 2540, 2877, 3609, 3016, 2345, 2926, 2228, 2508, 3237, 2853, 3386, 3270, 1997, 2405, 2934, 2573, 2623, 2583, 2846, 2890, 3152, 3081, 3836], 107334)


In [125]:
dl = datelist(start_month, end_month)
filepath=[]
for i in range(len(dl)):
    pathi = 'result0623/Full Brokerage/Morgan_Stanley/Morgan_Stanley'+dl[i]+'.json'
    filepath.append(pathi)

In [126]:
result = predict(DP.data,dl)

2017-06  completed
2017-07  completed
2017-08  completed
2017-09  completed
2017-10  completed
2017-11  completed
2017-12  completed
2018-01  completed
2018-02  completed
2018-03  completed
2018-04  completed
2018-05  completed
2018-06  completed
2018-07  completed
2018-08  completed
2018-09  completed
2018-10  completed
2018-11  completed
2018-12  completed
2019-01  completed
2019-02  completed
2019-03  completed
2019-04  completed
2019-05  completed
2019-06  completed
2019-07  completed
2019-08  completed
2019-09  completed
2019-10  completed
2019-11  completed
2019-12  completed
2020-01  completed
2020-02  completed
2020-03  completed
2020-04  completed
2020-05  completed


In [127]:
# run 1 time
for i in range(len(result)):
    with open(filepath[i],"w") as f:
        json.dump(result[i],f)

## UBS

In [128]:
start_month='2017-06'
end_month='2020-05'

DP=Data_Processor(start_month, end_month,
                  template=["Data/Full Brokerage/UBS/UBS"])
DP.readdata()
print(DP.datanums())
DP.specifylang()
DP.removenoise() 
DP.clean()
print(DP.datanums())

([9677, 9996, 9828, 9977, 9957, 10135, 9718, 9200, 10193, 10014, 9447, 9547, 9831, 8593, 10066, 9655, 10131, 9707, 9938, 9632, 9692, 9912, 9539, 9837, 9672, 9733, 9800, 10067, 10053, 10122, 9665, 9589, 9633, 10048, 10162, 10131], 352897)
([2296, 1901, 1910, 2412, 2608, 3479, 2604, 2073, 2236, 1916, 2216, 2487, 2757, 1891, 1983, 2263, 1882, 2207, 2016, 1786, 1941, 1996, 1933, 2012, 2283, 1858, 1555, 2282, 1792, 2056, 1808, 2052, 1908, 1778, 1736, 2065], 75978)


In [129]:
dl = datelist(start_month, end_month)
filepath=[]
for i in range(len(dl)):
    pathi = 'result0623/Full Brokerage/UBS/UBS'+dl[i]+'.json'
    filepath.append(pathi)

In [130]:
result = predict(DP.data,dl)

2017-06  completed
2017-07  completed
2017-08  completed
2017-09  completed
2017-10  completed
2017-11  completed
2017-12  completed
2018-01  completed
2018-02  completed
2018-03  completed
2018-04  completed
2018-05  completed
2018-06  completed
2018-07  completed
2018-08  completed
2018-09  completed
2018-10  completed
2018-11  completed
2018-12  completed
2019-01  completed
2019-02  completed
2019-03  completed
2019-04  completed
2019-05  completed
2019-06  completed
2019-07  completed
2019-08  completed
2019-09  completed
2019-10  completed
2019-11  completed
2019-12  completed
2020-01  completed
2020-02  completed
2020-03  completed
2020-04  completed
2020-05  completed


In [131]:
# run 1 time
for i in range(len(result)):
    with open(filepath[i],"w") as f:
        json.dump(result[i],f)